# Sklearn Pipeline for Scoring New Data

In [1]:
import psycopg2
import pickle
import pandas as pd

# Import Data
the dataset is available here: kaggle

In [2]:
## load data from local csv 
#df = pd.read_csv(
#        filepath_or_buffer = '../data/bank_customers_churn_dataset.csv',
#        index_col='customer_id'
#)
#
## drop churn column for segmentation
#df.drop('churn', axis=1, inplace=True)
#
## change type of categorical columns "credit_card" and "active_member"
#df['credit_card'] = df['credit_card'].apply(lambda x: 'yes' if x == 1 else 'no')
#df['active_member'] = df['active_member'].apply(lambda x: 'yes' if x == 1 else 'no')
#
#df.head()

In [3]:
def get_data_from_my_postgre_db(password:str, table_name:str)->pd.DataFrame:
    """connects to marketing_analytics db and returns data from table_name as pandas dataframe .
    inputs: database password and table_name"""
    # Connect to the database
    conn = psycopg2.connect(
        database="marketing_analytics", 
        user="postgres", 
        password=password, 
        host="localhost", 
        port="5432")
    # Create a cursor object
    cur = conn.cursor()
    # Execute a SQL query
    #cur.execute("SELECT * FROM bank_customers_churn_dataset")
    cur.execute("SELECT * FROM " + table_name) 
    # Get the column names from the cursor description
    columns = [desc[0] for desc in cur.description]
    # Fetch the results i.e. values
    results = cur.fetchall()
    # Create a dictionary mapping column names to values
    data = [dict(zip(columns, row)) for row in results]
    # Close the connection
    conn.close()
    # turn dictionary into dataframe
    return pd.DataFrame.from_dict(data)

In [4]:
f = open("../private.txt", "r")
pw = f.read()

In [5]:
# get data from postres db
table_name = "bank_customers_churn_dataset"
df = get_data_from_my_postgre_db(pw, table_name)

# set customer id as index
df.set_index('customer_id', inplace=True)

# drop churn column for segmentation
df.drop('churn', axis=1, inplace=True)

# change data types to numeric
df['tenure'] = df['tenure'].apply(lambda x: int(x))
df['products_number'] = df['products_number'].apply(lambda x: int(x))
df['credit_card'] = df['credit_card'].apply(lambda x: int(x))
df['active_member'] = df['active_member'].apply(lambda x: int(x))

# change type of categorical columns "credit_card" and "active_member"
df['credit_card'] = df['credit_card'].apply(lambda x: 'yes' if x == 1 else 'no')
df['active_member'] = df['active_member'].apply(lambda x: 'yes' if x == 1 else 'no')

df.head()

,credit_score,country,gender,age,tenure,balance,products_number,credit_card,active_member,estimated_salary
customer_id,,,,,,,,,,
15634602,619.0,France,Female,42,2,0.00,1,yes,yes,101348.88
15647311,608.0,Spain,Female,41,1,83807.86,1,no,yes,112542.58
15619304,502.0,France,Female,42,8,159660.80,3,yes,no,113931.57
15701354,699.0,France,Female,39,1,0.00,2,no,no,93826.63
15737888,850.0,Spain,Female,43,2,125510.82,1,yes,yes,79084.10


In [6]:
# Get one prediction for one row at a time
input_list = df.to_dict('records')

# select one row
data = input_list[:1]
data

[{'credit_score': 619.0,
  'country': 'France',
  'gender': 'Female',
  'age': 42,
  'tenure': 2,
  'balance': 0.0,
  'products_number': 1,
  'credit_card': 'yes',
  'active_member': 'yes',
  'estimated_salary': 101348.88}]

# Load Model Artifacts

In [7]:
with open('../artifacts/numeric_imputer.pickle', 'rb') as filename: # trained model to impute missing numeric data
    numeric_imputer = pickle.load(filename)

with open('../artifacts/categorical_imputer.pickle', 'rb') as filename: # trained model to impute missing categorical data
    categorical_imputer = pickle.load(filename) 

with open('../artifacts/rare_encoder.pickle', 'rb') as filename: # trained model to encode rare labels
    rare_encoder = pickle.load(filename)

with open('../artifacts/capper.pickle', 'rb') as filename: # trained model to cap outliers
    capper = pickle.load(filename)   

with open('../artifacts/enc.pickle', 'rb') as filename: # trained one hot encoder
    enc = pickle.load(filename)

with open('../artifacts/model.pickle', 'rb') as filename: # trained random forest classifier
    model = pickle.load(filename)

# Transform Dataset

In [8]:
# transform the row back to dataframe
df_row = pd.DataFrame(data)
df_row.head()

,credit_score,country,gender,age,tenure,balance,products_number,credit_card,active_member,estimated_salary
0,619.0,France,Female,42,2,0.0,1,yes,yes,101348.88


In [9]:
# get numeric and categorical columns
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
numeric_columns = df_row.select_dtypes(include=numerics).columns.to_list()
categorical_columns = df_row.select_dtypes(exclude=numerics).columns.to_list()

In [10]:
# impute mising numeric features
df_numeric = pd.DataFrame(
    numeric_imputer.transform(df_row[numeric_columns]), 
    columns=numeric_columns, 
    index=df_row.index)

# impute mising categorical features
df_categorical = pd.DataFrame(
    categorical_imputer.transform(df_row[categorical_columns]), 
    columns=categorical_columns, 
    index=df_row.index)

# concate numeric and categorical features
df_row = pd.concat([df_numeric, df_categorical], axis=1)

# remove rare labels
df_row[categorical_columns] = rare_encoder.transform(df_row[categorical_columns])

# remove outliers
df_row[numeric_columns] = capper.transform(df_row[numeric_columns])

# one hot encoding categorical features
df_cat_hotenc = pd.DataFrame(
    enc.transform(df_row[categorical_columns]), 
    columns=enc.get_feature_names_out(),
    index=df_row.index) 

# concate numeric and hot-encoded categorical features
df_hotenc = pd.concat([df_row[numeric_columns], df_cat_hotenc], axis=1)


df_row.head()

,credit_score,age,tenure,balance,products_number,estimated_salary,country,gender,credit_card,active_member
0,619.0,42.0,2.0,0.0,1.0,101348.88,France,Female,yes,yes


In [11]:
# predict cluster
labels = model.predict(df_hotenc)

# add cluster label to df
df_row['cluster'] = labels

df_row.head()

,credit_score,age,tenure,balance,products_number,estimated_salary,country,gender,credit_card,active_member,cluster
0,619.0,42.0,2.0,0.0,1.0,101348.88,France,Female,yes,yes,1


In [12]:
labels.tolist()

[1]

# Test on Fast API

In [13]:
import requests

In [14]:
url = 'http://127.0.0.1:8000/predict/'

In [15]:
data

[{'credit_score': 619.0,
  'country': 'France',
  'gender': 'Female',
  'age': 42,
  'tenure': 2,
  'balance': 0.0,
  'products_number': 1,
  'credit_card': 'yes',
  'active_member': 'yes',
  'estimated_salary': 101348.88}]

In [16]:
responses = []
for inputs in data:
    response = requests.post(
        url, 
        params=inputs)
    responses.append(response)

In [17]:
for response in responses:
    print(response.text)

{"predicted cluster label":[1]}
